In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
from scipy.misc import imread
import matplotlib.patches as patches
from torch.utils.data import DataLoader
from torch.optim import SGD, Adam
import cv2
from pycrayon import CrayonClient
from torch.utils.data import DataLoader
from faster_rcnn.network import  clip_gradient
import torch

def collate_fn(batch):
    return batch

In [3]:
import os
import torchvision.transforms as transforms
from faster_rcnn.utils.dataset import CocoData
from faster_rcnn.utils.data_generator import CocoGenerator
from faster_rcnn.utils.data_generator import Enqueuer

dataDir = './data/mscoco'
dataType = 'train2014'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)
# pre_proposal_folder = './data/mscoco/coco_proposals/MCG/'

images_dir = os.path.join(dataDir,'images', dataType)
cap = CocoData(root = images_dir,
                        annFile = annFile,
#                         pre_proposal_folder=pre_proposal_folder,
                        transform=transforms.ToTensor())



data_gen = CocoGenerator(data=cap)
queue = Enqueuer(generator=data_gen)
queue.start(max_queue_size=10, workers=2)
t = queue.get()


/data
/data/data/mscoco
loading annotations into memory...
Done (t=10.60s)
creating index...
index created!


In [4]:
from faster_rcnn.faster_rcnn import  RPN

In [5]:
cc = CrayonClient(hostname="crayon", port=8889)
try:
    exp = cc.create_experiment("RPN")
except: 
    cc.remove_experiment("RPN")
    exp = cc.create_experiment("RPN")

disp_interval = 100
log_interval = 10

In [6]:
net = RPN()
net.cuda()
net.train()

RPN (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU (inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU (inplace)
    (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU (inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU (inplace)
    (9): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU (inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU (inplace)
    (16): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), pa

In [7]:
param = filter(lambda x: x.requires_grad, net.parameters())

In [8]:
optimizer = SGD(param, lr=1e-4, momentum=0.9, weight_decay=0.0005)

In [9]:
def train(data_gen, optimizer, model, epoch):
    model.train()
    train_loss = 0
    losses = []
    for step in range(1, epoch):
#         blobs = data_gen.next()
        blobs = cap[13499]
        if blobs is None:
            continue
        if step > epoch:
            break
        im_data = blobs['data']
        im_info = blobs['im_info']
        gt_boxes = np.hstack([ blobs['boxes'] , blobs['gt_classes'][:, np.newaxis]])
        gt_ishard = blobs['gt_ishard']
        dontcare_areas = blobs['dontcare_areas']
        output = model(im_data, im_info, gt_boxes, gt_ishard, dontcare_areas)
        loss = model.loss
        print 'loss', loss
        train_loss += loss.data[0]
        optimizer.zero_grad()
        loss.backward()
        clip_gradient(model, 10.)
        optimizer.step()
        loss_value = loss.data.cpu().float()
        losses.append(loss_value)
        
        if step % 1 == 0:

            log_text = 'step %d, image: %s, loss: %.4f' % (
                step, blobs['im_name'], train_loss /(step))
            print(log_text)

            re_cnt = True

#         if step % log_interval == 0:
#             exp.add_scalar_value('loss', loss.data[0], step=step)
#             exp.add_scalar_value('last_layers_loss', net.loss.data[0], step=step)

In [10]:
train(t, optimizer, net, 10)

[[ 1.9788996   2.18660808  4.41185856  6.25701904]
 [-2.43394184 -5.72291327 -2.19459772  3.81811428]
 [ 1.81119823 -0.13457787  1.13333988  2.26057553]
 ..., 
 [ 0.21796353  4.62784815  1.43729854  3.76666594]
 [-0.26038569 -0.62148404 -0.41511074  1.4594512 ]
 [-4.65912867 -0.98074692  2.86140299  2.0340364 ]]
rpn_cls_score.shape (1, 18, 37, 50)
image info [[ 600.    800.      1.25]]
anchors:
[[ -84.  -40.   99.   55.]
 [-176.  -88.  191.  103.]
 [-360. -184.  375.  199.]
 [ -56.  -56.   71.   71.]
 [-120. -120.  135.  135.]
 [-248. -248.  263.  263.]
 [ -36.  -80.   51.   95.]
 [ -80. -168.   95.  183.]
 [-168. -344.  183.  359.]]
anchor shapes:
[[ 183.   95.]
 [ 367.  191.]
 [ 735.  383.]
 [ 127.  127.]
 [ 255.  255.]
 [ 511.  511.]
 [  87.  175.]
 [ 175.  351.]
 [ 351.  703.]]
AnchorTargetLayer: height 37 width 50

im_size: (600.0, 800.0)
scale: 1.25
height, width: (37, 50)
rpn: gt_boxes.shape (3, 5)
rpn: gt_boxes [[ 140.    253.75  185.    430.     24.  ]
 [ 318.75  247.5   442.5

step 5, image: COCO_train2014_000000309241.jpg, loss: 207.9302
[[ 2.01664066  2.14559746  4.32283592  6.16395473]
 [-2.40709615 -5.65244865 -2.16751719  3.79762363]
 [ 1.88160038 -0.21351023  1.09861791  2.31298995]
 ..., 
 [ 0.13408156  4.71459627  1.39773202  3.66902113]
 [-0.14807826 -0.56031293 -0.45720699  1.44443738]
 [-4.67316961 -1.05764604  2.82506323  2.15405273]]
rpn_cls_score.shape (1, 18, 37, 50)
image info [[ 600.    800.      1.25]]
anchors:
[[ -84.  -40.   99.   55.]
 [-176.  -88.  191.  103.]
 [-360. -184.  375.  199.]
 [ -56.  -56.   71.   71.]
 [-120. -120.  135.  135.]
 [-248. -248.  263.  263.]
 [ -36.  -80.   51.   95.]
 [ -80. -168.   95.  183.]
 [-168. -344.  183.  359.]]
anchor shapes:
[[ 183.   95.]
 [ 367.  191.]
 [ 735.  383.]
 [ 127.  127.]
 [ 255.  255.]
 [ 511.  511.]
 [  87.  175.]
 [ 175.  351.]
 [ 351.  703.]]
AnchorTargetLayer: height 37 width 50

im_size: (600.0, 800.0)
scale: 1.25
height, width: (37, 50)
rpn: gt_boxes.shape (3, 5)
rpn: gt_boxes [[ 1

In [ ]:
torch.save(net.state_dict(), 'rpn.pkl')

In [ ]:
net.load_state_dict(torch.load('./rpn.pkl'))


In [ ]:
blobs =  cap[13499]
im_data = blobs['data']
im_info = blobs['im_info']
gt_boxes = np.hstack([ blobs['boxes'] , blobs['gt_classes'][:, np.newaxis]])
gt_ishard = blobs['gt_ishard']
dontcare_areas = blobs['dontcare_areas']

print blobs['boxes'] / im_info[0][2]
print blobs['image_info']
print im_data.shape
origin_gt_box =  blobs['boxes'] / im_info[0][2]
print origin_gt_box
print gt_boxes

In [ ]:
# net.load_state_dict(torch.load('./rpn.pkl'))

net.eval()
_, result = net(im_data, im_info, gt_boxes, gt_ishard, dontcare_areas)

In [ ]:
box_data =  result.data.cpu().numpy()
features =  _.data.cpu().numpy()
print features.shape
print box_data.shape

In [ ]:
fig,ax = plt.subplots(1)
base_path = './data/mscoco/images/train2014/'
img_path = os.path.join(base_path, blobs['im_name'])
# Display the image
im_data = cv2.imread(img_path)

ax.imshow(im_data)
# Create a Rectangle patch
for i, box in enumerate(box_data):
    box = box[1:]
    rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1] ,linewidth=1,edgecolor='b',facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)
    
for i, box in enumerate(origin_gt_box):
    rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1]  ,linewidth=1,edgecolor='r',facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)
    
plt.show()

# debug max overlap